In [31]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [32]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response
from multimodal import MultiModal

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="15.Agent")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 15.Agent]


## 도구(Tools)

[Langchain 통합된 도구](https://python.langchain.com/v0.1/docs/integrations/tools/)  
- Langchain에 통합된 다양한 기능들이 있습니다. 이곳의 내용들 중에 필요한 것들을 사용합니다.  
- 이번에는 `Langchain 통합된 도구` 에서 도구들을 가져와 사용하는 방법을 살펴봅니다.

### 빌트인 도구 (built-in tools)
랭체인에서 제공하는 사전에 정의된 도구(tools) 와 툴킷(toolkit) 을 사용할 수 있습니다.  
tool 은 단일 도구를 의마하며, toolkit은 여러 도구를 묶어서 하나의 도구로 사용할 수 있습니다.   
  
도구 링크  
- https://python.langchain.com/docs/integrations/tools/

### 유용한 도구들

#### 1. Python REPL Tool

이 Tool 은 Python 코드를 REPL(Read-Eval_Print Loop) 환경에서 실행하기 위한 클래스를 제공합니다.  
클래스 이름은 `PythonREPLTool` 입니다.  

**설명**
- Python Cell 환경을 제공합니다.
- Python 코드를 실행할 수 있습니다.
- 결과는 `print()` 메서드로 확인 할 수 있습니다.

**주요 특징**
- `sanitize_input`: 입력을 정제하는 옵션입니다. (derault:True)
- `run()`, `aerun()`, `invoke()` 메서드를 사용하여 Python 코드를 실행합니다.

**입력 정제**
- 입력된 문자열에서 불필요한 공백, 백틱, `python 언어에서 지원하는 키워드` 등을 제거합니다.

In [2]:
from langchain_experimental.tools import PythonREPLTool

# 도구 생성
python_repl_tool = PythonREPLTool()

In [3]:
# 파이썬 코드 실행 후 입력 반환
python_code = "print(100 + 200)"
print(python_repl_tool.invoke(python_code))

Python REPL can execute arbitrary code. Use with caution.


300



아래는 LLM 에게 Python 코드를 작성하도록 요청하고 결과를 반환하는 예제입니다.  
1. LLM 모델에게 Python 코드를 작성하도록 요청합니다.
2. 작성된 코드를 실행하여 결과를 얻습니다.
3. 결과를 출력합니다.

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_experimental.tools import PythonREPLTool


# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    # 도구 생성
    python_tool = PythonREPLTool()
    if debug:
        print("CODE: \n" + code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하는 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no main, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 체인 생성, StrOutputParser() 까지 코드가 오면 그것을 print_and_excute의 인자로 입력하게 됨
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

# 확인
print(chain.invoke({"input": "로또 번호를 생성하고 출력하는 코드를 만들어주세요."}))

CODE: 
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())
[2, 35, 38, 39, 41, 45]



In [30]:
import random


def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))


print(generate_lotto_numbers())

[5, 8, 11, 29, 31, 32]


#### 2. 검색 API Tool

Tavily 검색 API를 활용하면 인터넷으로 외부검색을 할 수 있습니다.    
두가지 클래스를 사용합니다.   
`TavilySearchResults`, `TavilyAnswer` 입니다.  
  
API KEY 를 발급받아야 합니다.(한 달에 1000번의 요청은 무료)    
- 링크 : https://app.tavily.com

발급받은 API KEY 는 .env 파일에 `TAVILY_API_KEY` 로 설정합니다.

**TavilySearchResults**
- Tavily 검색 API 를 쿼리하고 JSON 형태로 반환합니다.
- 포괄적이고 정확하며 신뢰할수 있는 결과에 최적화된 엔진입니다.
- 현재 이벤트에 대한 질문에 답변할 때 유용합니다.

주요 매개 변수
- max_results (int): 반환할 최대 검색 결과 수 (기본값: 5)
- search_depth (str): 검색 깊이 ("basic" 또는 "advanced")
- include_domains (List[str]): 검색 결과에 포함할 도메인 목록
- exclude_domains (List[str]): 검색 결과에서 제외할 도메인 목록
- include_answer (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- include_raw_content (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- include_images (bool): 쿼리 관련 이미지 목록 포함 여부

반환 값
- 검색 결과를 JSON 형식의 문자열(url, content)

In [35]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tavily_tool = TavilySearchResults(
    max_results=3,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []
)

In [36]:
# 도구 실행
tavily_tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://wikidocs.net/262582',
  'content': "LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다. LangChain 한국어 튜토리얼\\n바로가기 👀\\n[LangChain] 에이전트(Agent)와 도구(tools)를 활용한 지능형 검색 시스템 구축 가이드\\n2024년 02월 09일\\n41 분 소요\\n이 글에서는 LangChain 의 Agent 프레임워크를 활용하여 복잡한 검색과 📍 전체 템플릿 코드\\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\\nLangSmith 추적\\n마무리입니다!\\n 문서 기반 QA 시스템 설계 방법 - 심화편\\n2024년 02월 06일\\n22 분 소요\\nLangChain의 RAG 시스템을 통해 문서(PDF, txt, 웹페이지 등)에 대한 질문-답변을 찾는 과정을 정리하였습니다.\\n'},  {'url': 'https://wikidocs.net/234282',  'content': 'Mar 19, 2024 · langchain 은 언어 모델과 관련된 다양한 기능을 제공하는 라이브러리로, 이 중 검색 도구 생성 기능은 데이터 검색 및 처리 작업을 용이하게 한다.'}] 이 도구를 사용하면 DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능을 구현할 수 있습니다."},
 {'url': 'https://teddylee777.github.io/langchain/langchain-agent/',
  'content': '태그:\nAgent,\nAPI KEY,\nFAISS,\nLangChain,\nLangSmith,\nmemory,\nOpenAI,\nPython,\nRetriever,\nTavily Search,\ntools,\n검색도구,\n랭체인,\n에이전트\n카테고리:\nlangchain\n업데이트: 2024년 02월 09일\n참고\n

### 사용자 정의 도구(Custom Tool)

LangChain 에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용할 수 있습니다.  
이를 위해서는 langchain.tools 모듈에서 제공하는 tool 데코레이터를 사용하여 함수를 도구로 변환합니다.  
  
`@tool` 데코레이터
- 함수를 도구로 변환하는 기능을 제공합니다. 다양한 옵션을 통해 도구의 동작을 커스터마이즈할 수 있습니다.  
  
사용 방법  
- 함수 위에 @tool 데코레이터 적용  
- 필요에 따라 데코레이터 매개변수 설정

이 데코레이터를 사용하면 일반 Python 함수를 강력한 도구로 쉽게 변환할 수 있으며, 자동화된 문서화와 유연한 인터페이스 생성이 가능합니다.